In [85]:
import os
from ast import literal_eval
import pandas as pd
from pathlib import Path
from environ import Env
import warnings
warnings.filterwarnings('ignore')

# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(os.curdir).resolve().parent.parent
env = Env()
env_path = BASE_DIR / "django/.env"
if env_path.exists():
    with env_path.open("rt", encoding="utf8") as f:
        env.read_env(f, overwrite=True)

dbname = env.get_value('GCPDB_NAME')
user = env.get_value('GCPDB_USER')
pw = env.get_value('GCPDB_PASSWORD')
host = env.get_value('GCPDB_HOST')

from sqlalchemy import create_engine
# echo를 true로 설정하면 command창에 실행된 sql문이 뜨게 됨
engine = create_engine(f'mysql+mysqldb://{user}:{pw}@{host}:3306/{dbname}?charset=utf8')

PKL_DIR = Path(os.curdir).resolve().parent.parent
PKL_DIR = PKL_DIR / "Utils/Pickle"

In [23]:
!ls ../../Data/EDA/ml-25m/

README.txt	   genome-tags.csv  movies.csv	 tags.csv
genome-scores.csv  links.csv	    ratings.csv


In [11]:
!ls /opt/ml/project2/Utils/Pickle/23*

/opt/ml/project2/Utils/Pickle/230130_Popular_movie_1192_cwj.pickle
/opt/ml/project2/Utils/Pickle/230130_Popular_movie_character_2867_cwj.pickle


In [24]:
rating = pd.read_csv('/opt/ml/project2/Data/EDA/ml-25m/ratings.csv')

In [12]:
cdf = pd.read_pickle(PKL_DIR / '230130_Popular_movie_character_2867_cwj.pickle')
mdf = pd.read_pickle(PKL_DIR / '230130_Popular_movie_1192_cwj.pickle')

## Fake User DB에서 불러오기

In [36]:
tmp_user = pd.read_sql_query("select * from test_rec_tmpuser", engine)
auth_user = pd.read_sql_query("select * from auth_user", engine)

In [68]:
auth_cols=['id','username']
tmp_cols = ['LoginUser_id','MBTI','ennea_res','prefer_movie_id','create_time']

In [69]:
fake_user = auth_user[auth_user.id==2]
fake_user

,id,password,last_login,is_superuser,username,first_name,last_name,email,is_staff,is_active,date_joined
1,2,!mmQE7zcvRQ9pcGnOtyEMySzGF8peQu62SKVirA6o,2023-01-29 18:14:17.479271,0,awsed2486,나현,이,awsed2486@gmail.com,0,1,2023-01-29 18:14:11.020115


In [77]:
fake_user_df = fake_user[auth_cols].merge(tmp_user[tmp_cols], left_on='id',right_on='LoginUser_id')
fake_user_df.prefer_movie_id = fake_user_df.prefer_movie_id.map(literal_eval)
fake_user_df

,id,username,LoginUser_id,MBTI,ennea_res,prefer_movie_id,create_time
0,2,awsed2486,2.0,ESTP,1w2,"[103228, 5445, 48516, 3535, 44199]",2023-01-29 18:12:49.103621
1,2,awsed2486,2.0,ISTP,1w2,"[64614, 62155, 55247, 164909, 64969, 112552, 6...",2023-01-30 01:49:01.041521
2,2,awsed2486,2.0,ENTP,1w2,"[31878, 72011, 5620, 3996, 41571, 4447]",2023-01-30 04:22:55.664887
3,2,awsed2486,2.0,ENTP,1w2,"[8949, 105844, 6711, 8666, 8798, 54995]",2023-01-30 05:02:00.143908
4,2,awsed2486,2.0,ENFJ,4w5,"[30749, 132480, 194448, 44195, 4014, 8376]",2023-01-30 13:20:28.742866


### MBTI, ennea_res가 바뀐 다면 create_time이 가장 마지막인 것을 기준으로 추천하기

## 기존 mdf와 MovieLens로 Interaction 만들기

In [65]:
mdf_rating = rating[rating.movieId.isin(mdf.movieId)]
mdf_rating

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
9,1,1250,4.0,1147868414
11,1,1653,4.0,1147868097
23,1,3949,5.0,1147868678
25,1,4308,3.0,1147868534
...,...,...,...,...
25000081,162541,7361,4.5,1240953484
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697


In [86]:
mdf_rating.rating=1
mdf_rating[:2]

,userId,movieId,rating,timestamp
0,1,296,1,1147880044
9,1,1250,1,1147868414


## Fake User의 Interaction 합치기

### Fake User의 userID 변환 (Django DB user id: rating에 쓰일 user id)

In [67]:
mdf_rating.userId.max()

162541

In [104]:
common_cols = ['userId','movieId']

In [105]:
fake_user_inter = fake_user_df[['id','prefer_movie_id']].explode('prefer_movie_id')
# fake_user_inter['rating']=1
fake_user_inter.columns=common_cols
print(f"유저별 중복 영화 제거 전: {fake_user_inter.shape[0]}")
fake_user_inter.drop_duplicates(subset=['userId','movieId'], inplace=True)
print(f"유저별 중복 영화 제거 후: {fake_user_inter.shape[0]}")
fake_user_inter.userId+=300000
fake_user_inter[:10]

유저별 중복 영화 제거 전: 41
유저별 중복 영화 제거 후: 41


,userId,movieId
0,300002,103228
0,300002,5445
0,300002,48516
0,300002,3535
0,300002,44199
1,300002,64614
1,300002,62155
1,300002,55247
1,300002,164909
1,300002,64969


### 합치기

In [106]:
mdf_rating.shape, fake_user_inter.shape

((6348460, 4), (41, 2))

In [107]:
merge_rating = pd.concat([mdf_rating[common_cols],fake_user_inter])
merge_rating.shape

(6348501, 2)

In [108]:
merge_rating.columns = ['userId:token', 'movieId:token']
merge_rating[:3]

,userId:token,movieId:token
0,1,296
9,1,1250
11,1,1653


## Recbole input 만들기 (.inter, .data 등)

In [114]:
!ls ../../BaseLine/dataset/train_data/
!readlink -ef ../../BaseLine/dataset/train_data/

train_data.inter
/opt/ml/project2/BaseLine/dataset/train_data


In [115]:
merge_rating.to_csv("/opt/ml/project2/BaseLine/dataset/train_data/train_data.inter",sep='\t',index=False)

## EASE 학습 및 추천

In [ ]:
## 추천 movieids 추출

In [ ]:
## DB에 저장

In [117]:
fake_user_inter.to_sql('table_name',engine)

41

In [ ]:
## TmpUser와 User의 관계가 어떻게 되는지, 어떻게 추천을 해줄 수 있는지 내일 나현님께 여쭤보기
# -> 답변완료